# Control moment gyroscope

Import modules and configure the notebook.

In [1]:
# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

# This is my own script (it is an interface to the pybullet simulator)
import ae353_cmg

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae353_cmg)

<module 'ae353_cmg' from 'C:\\Users\\david\\Documents\\uiuc\\ae353\\ae353-sp22\\projects\\01_cmg\\ae353_cmg.py'>

Create an instance of the `Simulator` class, which is an interface to the [pybullet](http://pybullet.org) simulation engine.

There are several optional arguments:

* `display` (`True` or `False`) is whether or not to show the simulation window;
* `damping` (a non-negative floating-point number, default `0.`) is the coefficient of viscous friction at each joint;
* `load_mass` (a non-negative floating-point number, default `1.`) is the mass of the sphere attached to the platform;
* `roll` (an angle in radians, default `np.pi / 2`) is the roll angle of the ground.

**You must evaluate this cell only *once*.** If you want to start fresh with a new simulator, you must do `Kernel -> Restart` from the notebook menu first. If you evaluate this cell more than once without a call to `Kernel -> Restart` then you may get strange behavior that is hard to debug.

In [2]:
simulator = ae353_cmg.Simulator(
    display=True,
    damping=0.,
    load_mass=1.,
    roll=(np.pi / 2),
)

Define a controller that maps sensor measurements to actuator commands. By default, it chooses to apply zero torque to the gimbal. Modify the `run` function to make the controller behave differently.

In [3]:

#find the A and B
v3 = 1000
q1,q2,v1,v2,tau = sym.symbols('q1,q2,v1,v2,tau')
f = sym.Matrix([v1,v2,((-39.24*sym.sin(q1) + 0.01*sym.sin(2*q2)*v1*v2 + 0.02*sym.cos(q2)*v2*v3)/(2*(0.01*sym.cos(q2)**2 + 4.501))),(90.9090909090909*tau - 0.909090909090909*sym.cos(q2)*v1*v3)])

q1_e = 2*np.pi
q2_e = 0.
v1_e = 0.
v2_e = 0.
tau_e = 0.

f1_num = sym.lambdify([q1,q2,v1,v2,tau],f)
f1_num(q1_e, q2_e, v1_e, v2_e, tau_e)
A_num = sym.lambdify((q1,q2,v1,v2,tau), f.jacobian([q1,q2,v1,v2]))
A=A_num(q1_e, q2_e, v1_e, v2_e, tau_e)

B_num = sym.lambdify([q1,q2,v1,v2,tau], f.jacobian([tau]),'numpy')
B=B_num(q1_e, q2_e, v1_e, v2_e, tau_e)
print(A)
print(B)


class Controller:
    def __init__(self, K, q1_e, q2_e, v1_e, v2_e, tau_e):
        self.K = K
        self. q1_e = q1_e
        self. q2_e = q2_e
        self. v1_e = v1_e
        self. v2_e = v2_e
        self. tau_e = tau_e
    
    def reset(self):
        pass
    
    def run(
            self,
            t,
            platform_angle,
            platform_velocity,
            gimbal_angle,
            gimbal_velocity,
            rotor_velocity
        ):
        
        #Find state
        x = np.array([platform_angle - self.q1_e, 
                      platform_velocity - self.v1_e, 
                     gimbal_angle -self.q2_e, 
                     gimbal_velocity - self.v2_e, 
                     ])
        #Find input
        u = -self.K@x
        # FIXME: replace the following line
        gimbal_torque = 2
        
        return gimbal_torque

[[   0.            0.            1.            0.        ]
 [   0.            0.            0.            1.        ]
 [  -4.34936821    0.            0.            2.21680337]
 [   0.            0.         -909.09090909    0.        ]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [90.90909091]]


Create an instance of the controller.

In [4]:
controller = Controller()

TypeError: __init__() missing 6 required positional arguments: 'K', 'q1_e', 'q2_e', 'v1_e', 'v2_e', and 'tau_e'

Reset the simulator with given initial conditions. A low-level controller (separate from yours) will try to ensure that the rotor velocity remains constant.

In [ ]:
simulator.reset(
    platform_angle=0.,
    platform_velocity=0.,
    gimbal_angle=0.,
    gimbal_velocity=0.,
    rotor_velocity=1000.,
)

Reset the controller.

In [ ]:
controller.reset()

Choose a camera view (could be `simulator.camera_topview()` or `simulator.camera_sideview()`).

In [ ]:
simulator.camera_sideview()

Run the simulator.

In [ ]:
data = simulator.run(
    controller,           # <-- required (an instance of your Controller class)
    max_time=10.0,         # <-- optional (how long you want to run the simulation in seconds)
    data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
    video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
)

Get, show, and save a snapshot of the simulation after the last time step.

In [ ]:
# Get snapshot as height x width x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(8, 8))
plt.imshow(rgba)

# Save snapshot
plt.imsave('my_snapshot.png', rgba)

Plot the results.

In [ ]:
# Create a figure with three subplots, all of which share the same x-axis
fig, (ax_platform, ax_gimbal, ax_rotor, ax_torque) = plt.subplots(4, 1, figsize=(8, 8), sharex=True)

# Plot platform angle and velocity
ax_platform.plot(
    data['t'], data['platform_angle'],
    label='platform angle (rad)', linewidth=4,
)
ax_platform.plot(
    data['t'], data['platform_velocity'],
    '--', label='platform velocity (rad / s)', linewidth=4,
)
ax_platform.grid()
ax_platform.legend(fontsize=16)
ax_platform.tick_params(labelsize=14)

# Plot gimbal angle and velocity
ax_gimbal.plot(
    data['t'], data['gimbal_angle'],
    label='gimbal angle (rad)', linewidth=4,
)
ax_gimbal.plot(
    data['t'], data['gimbal_velocity'],
    '--', label='gimbal velocity (rad / s)', linewidth=4,
)
ax_gimbal.grid()
ax_gimbal.legend(fontsize=16)
ax_gimbal.tick_params(labelsize=14)

# Plot rotor velocity
ax_rotor.plot(
    data['t'], data['rotor_velocity'],
    '--', label='rotor velocity (rad / s)', linewidth=4,
)
ax_rotor.grid()
ax_rotor.legend(fontsize=16)
ax_rotor.tick_params(labelsize=14)
ax_rotor.set_ylim(simulator.rotor_velocity - 10., simulator.rotor_velocity + 10.)

# Plot torques
ax_torque.plot(
    data['t'], data['gimbal_torque_command'],
    label='gimbal torque command (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], data['gimbal_torque'],
    '--', label='gimbal torque (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], data['rotor_torque'],
    '--', label='rotor torque (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], np.ones_like(data['t']) * simulator.tau_max,
    ':', label='max torque', linewidth=4, color='C4', zorder=0,
)
ax_torque.plot(
    data['t'], -np.ones_like(data['t']) * simulator.tau_max,
    ':', linewidth=4, color='C4', zorder=0,
)
ax_torque.grid()
ax_torque.legend(fontsize=16)
ax_torque.tick_params(labelsize=14)

# Set x-axis properties (only need to do this on the last
# subplot since all subplots share the same x-axis)
ax_torque.set_xlabel('time (s)', fontsize=20)
ax_torque.set_xlim([data['t'][0], data['t'][-1]])
ax_torque.set_ylim(-1.2 * simulator.tau_max, 1.2 * simulator.tau_max)

# Make the arrangement of subplots look nice
fig.tight_layout()

Save the figure (without transparent background).

In [ ]:
fig.savefig('my_figure.png', facecolor='white', transparent=False)